<a href="https://colab.research.google.com/github/annvorosh/GB/blob/ETL/ETL_L07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL: автоматизация подготовки данных (семинары)
## Урок 7. Построение пайплайнов и визуализация потоков данных в Airflow
— Зарегистрируйтесь в ОрепWeatherApi (https://openweathermap.org/api)
— Создайте ETL, который получает температуру в заданной вами локации, и
дальше делает ветвление:

• В случае, если температура больше 15 градусов цельсия — идёт на ветку, в которой есть оператор, выводящий на
экран «тепло»;
• В случае, если температура ниже 15 градусов, идёт на ветку с оператором, который выводит в консоль «холодно».

Оператор ветвления должен выводить в консоль полученную от АРI температуру.

— Приложите скриншот графа и логов работы оператора ветвленния.

In [1]:
!pip install apache-airflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.3/511.3 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 4.5 MB/s 

In [2]:
!pip install requests

In [3]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
import requests

# Определение даты начала выполнения DAG
start_date = datetime(2024, 1, 1)

# Определение интервала выполнения DAG
schedule_interval = timedelta(days=1)

# Создание объекта DAG
dag = DAG(
    'weather_etl',
    start_date=start_date,
    schedule_interval=schedule_interval,
)

# Функция для получения температуры по API
def get_temperature(location):
    response = requests.get(f'https://api.openweathermap.org/data/2.5/weather?q={location}&appid=c5e2524ab03a711ea93b95ef76fead96')
    data = response.json()
    temperature = data['main']['temp']
    return temperature

# Функция для вывода сообщения в консоль в зависимости от температуры
def print_temperature_message(**kwargs):
    location = kwargs.get('location', 'Vyborg')
    temperature = kwargs['ti'].xcom_pull(task_ids='get_temperature')

    if temperature > 15:
        print(f"Weather in {location}: {temperature}°C - warm")
    else:
        print(f"Weather in {location}: {temperature}°C - cold")

# Оператор для получения температуры
get_temperature_operator = PythonOperator(
    task_id='get_temperature',
    python_callable=get_temperature,
    op_kwargs={'location': 'Vyborg'},
    provide_context=True,
    dag=dag,
)

# Оператор ветвления
branch_operator = PythonOperator(
    task_id='branch_operator',
    python_callable=print_temperature_message,
    provide_context=True,
    dag=dag,
)

# Установка зависимостей между операторами
get_temperature_operator >> branch_operator


<ipython-input-3-3411f0109126>:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

<ipython-input-3-3411f0109126>:13 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

<ipython-input-3-3411f0109126>:37 RemovedInAirflow3Warning: provide_context is deprecated as of 2.0 and is no longer required

<ipython-input-3-3411f0109126>:46 RemovedInAirflow3Warning: provide_context is deprecated as of 2.0 and is no longer required

<Task(PythonOperator): branch_operator>